IMPORTAÇÃO DE BIBLIOTECAS E DEFINIÇÃO DE VARIÁVEIS

In [ ]:
import numpy as np
import random
import pandas as pd
import glob
from smt.sampling_methods import LHS
import matplotlib.pyplot as plt

path_input = r"C:\Documentos\Modelo Base\UNISIM-I-D.dat"
path_output = r"C:\Documentos\Modelos\{}. UNISIM-I-D- {} - {} - {} - {} - {}.dat"
path_logs = r'C:\Documentos\logs' #caminho da pasta com os logs
path_result = r'C:\Documentos\Resultado\resultado.xlsx'
num_modelos = 500
# DWOC, COMPRESSIBILIDADE, MULTIPLICADOR DE PERMEABILIDADE, DENSIDADE DO ÓLEO, TRANSMISSIBILIDADE
xlimits = np.array([[3074, 3274], [0.0000236, 0.0000824], [0.475,2.525], [750,900], [0,1]])
sampling = LHS(xlimits=xlimits)
x=sampling(num_modelos)

df = pd.DataFrame()
df_dwoc = []
df_comp = []
df_multi = []
df_dens = []
df_trans = []
count = 0

PROCESSO ITERATIVO DE GERAÇÃO DE MODELOS

In [ ]:
for i in range(0,num_modelos): #processo iterativo de geração de  modelos
    df_dwoc.append(x[i][0])
    df_comp.append(x[i][1])
    df_multi.append(x[i][2])
    df_dens.append(x[i][3])
    df_trans.append(x[i][4])
    with open(path_input, 'r') as file : #processo de leitura do modelo base
        filedata = file.read()
    filedata = filedata.replace('DWOC    3100.0  3174.0', 'DWOC  3100.0  {}'.format(x[i][0]))#substituição dos valores do modelo base
    filedata = filedata.replace('DATUMDEPTH 3174.0 INITIAL', 'DATUMDEPTH {} INITIAL'.format(x[i][0]))#substituição dos valores do modelo base
    filedata = filedata.replace('DENSITY OIL 0.866E3', 'DENSITY OIL {}'.format(x[i][3]))#substituição dos valores do modelo base
    filedata = filedata.replace('CPOR 53.0E-6', 'CPOR {}'.format(x[i][1]))#substituição dos valores do modelo base
    filedata = filedata.replace('*MOD * 1.5', '*MOD * {}'.format(x[i][2]))#substituição dos valores do modelo base
    filedata = filedata.replace("TRANSF 'f3'    0.000", "TRANSF 'f3'    {}".format(x[i][4]))#substituição dos valores do modelo base
    x
    with open(path_output.format(count,x[i][0],x[i][1],x[i][2],x[i][3],x[i][4]), 'w') as file:
      file.write(filedata) #salvamento do modelo modificado
    count+=1
df['DWOC 1'] = df_dwoc #criação de tabelas para armazenamento das informações de geração
df['COMPRESSIBILIDADE'] = df_comp #criação de tabelas para armazenamento das informações de geração
df['MULTIPLICADOR'] = df_multi #criação de tabelas para armazenamento das informações de geração
df['DENSIDADE OLEO'] = df_dens #criação de tabelas para armazenamento das informações de geração
df['TRANSMISSIBILIDADE'] = df_trans #criação de tabelas para armazenamento das informações de geração

df['DWOC 1'] = df['DWOC 1'].astype(str).str.strip('[]') #transformação de dados para armazenamento no dataframe
df['COMPRESSIBILIDADE'] = df['COMPRESSIBILIDADE'].astype(str).str.strip('[]')
df['MULTIPLICADOR'] = df['MULTIPLICADOR'].astype(str).str.strip('[]')
df['DENSIDADE OLEO'] = df['DENSIDADE OLEO'].astype(str).str.strip('[]')
df['TRANSMISSIBILIDADE'] = df['TRANSMISSIBILIDADE'].astype(str).str.strip('[]')
df['simulação'] = df.index

LEITURA DE LOGS E CAPTAÇÃO DE RESULTADOS

In [ ]:
all_files = glob.glob(path_logs + "/*.log") #leitura de todos os arquivos
dict_lista = {}
li = []
li_num = []
li_2 = []
count = 0
variaveis = ['Current oil in place', 'Original oil in place'] #variáveis a serem exploradas no arquivo log

for variavel in variaveis: #processo iterativo para cada variável
    dict_lista[str(variavel)] = []
    for filename in all_files:
        if count == 0:
            li_num.append(filename.split('\\')[5].split('. ')[0])
        with open(filename,'r') as file:
            for line in reversed(file.readlines()):
                if variavel in line:
                    print(line)
                    dict_lista[str(variavel)].append(line.replace(' ', '').split('|')[2])
                    print(dict_lista)
                    break
    count+=1

ORGANIZAÇÃO DOS RESULTADOS E EXPORTAÇÃO DO RESULTADO

In [ ]:
df_novo = pd.DataFrame()
df_novo['simulação'] = li_num
df_novo['Current oil in place'] = dict_lista.get('Current oil in place')
df_novo['Original oil in place'] = dict_lista.get('Original oil in place')
df_novo['Original water in place'] = dict_lista.get('Original water in place')
df_novo['Original gas in place'] = dict_lista.get('Original gas in place')
df_novo['Current water in place'] = dict_lista.get('Current water in place')
df_novo['Current gas in place'] = dict_lista.get('Current gas in place')
df_novo['Fator de recuperação do óleo (%)'] = dict_lista.get('Oil recovery factor')

df_novo['simulação'] = df_novo['simulação'].astype(int)
df_final = df.merge(df_novo, left_on='simulação', right_on='simulação')
df_final['Produção de óleo'] = df_final['Original oil in place'].astype(float) - df_final['Current oil in place'].astype(float)
df_final['Produção de água'] = df_final['Original water in place'].astype(float) - df_final['Current water in place'].astype(float)
df_final['Produção de gás'] = df_final['Original gas in place'].astype(float) - df_final['Current gas in place'].astype(float)
df_final.to_excel(path_result)